In [1]:
import os
import shutil
from seeq import spy

In [2]:
# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../credentials.key', force=False)

# spy.workbooks

The `spy.workbooks` module provides functions for importing and exporting _workbooks_. A workbook is either a _Workbench Analysis_ (colored green in the Seeq user interface) or an _Organizer Topic_ (colored blue).

This functionality is helpful to:

- Move content between two Seeq servers
- Manage content by exporting and committing to a version control system like Git

The process typically starts by searching for some content that you have created in Seeq and exporting it. However, since this documentation needs some pre-built content to illustrate how it works, there is a pre-built-and-exported set of workbooks alongside this documentation notebook. So we'll go in a non-typical order of operations in this example.

## The Export Format

When content is exported from Seeq, each workbook is encapsulated in its own folder, including all its worksheets, calculated item definitions and all dependencies, journal/document images and anything else that is necessary to import that data into another server. Content is written to disk as either JSON, HTML or image files as appropriate. References to datasource items are also catalogued during export and default _datasource maps_ are created that facilitate identification of equivalent signals/conditions/scalars on the destination system so that the imported content gets "hooked up" to the right data.

## Main Actions

There are five main operations you can perform on workbooks:

- **search** for workbooks whose content you want to *pull*
- **pull** those workbooks into `Workbook` in-memory Python objects
- **save** the `Workbook` Python objects to disk in the export format described above
- **load** `Workbook` Python objects from disk into memory
- **push** in-memory `Workbook` Python objects into a Seeq Server

As mentioned, we're going to go out-of-order for illustration purposes: _load_, _push_, _search_, _pull_, _save_.

### Importing

This set of documentation comes with an _Example Export_ folder that contains an Analysis and a Topic for illustration purposes. First we **load** it into memory:

In [3]:
workbooks = spy.workbooks.load('Example Export.zip')
workbooks

[Workbook "Example Analysis" (D833DC83-9A38-48DE-BF45-EB787E9E8375),
 Workbook "Example Topic" (811B1488-297A-4FD2-AE7C-A1FE0E3B3641)]

Now that the workbook definitions are in memory, we can push them into Seeq.

In [4]:
spy.workbooks.push(workbooks, path='My Import', errors='raise')

,ID,Name,Type,Workbook Type,Count,Time,Result,Pushed Workbook ID,URL
0,D833DC83-9A38-48DE-BF45-EB787E9E8375,Example Analysis,Workbook,Analysis,51,00:00:01.36,Success,B70DAB96-988C-452E-9F91-3977802EE2D4,link
1,811B1488-297A-4FD2-AE7C-A1FE0E3B3641,Example Topic,Workbook,Topic,6,00:00:00.15,Success,7EF60B3D-1836-4728-8A9E-6CFB9000B609,link


,ID,Name,Type,Workbook Type,Count,Time,Result,Pushed Workbook ID,URL
0,D833DC83-9A38-48DE-BF45-EB787E9E8375,Example Analysis,Workbook,Analysis,51,0:00:01.360896,Success,B70DAB96-988C-452E-9F91-3977802EE2D4,http://localhost:34216/EC566060-7D5D-421A-870C...
1,811B1488-297A-4FD2-AE7C-A1FE0E3B3641,Example Topic,Workbook,Topic,6,0:00:00.151590,Success,7EF60B3D-1836-4728-8A9E-6CFB9000B609,http://localhost:34216/EC566060-7D5D-421A-870C...


The workbooks have been imported into Seeq in a _My Import_ folder with you as the owner. Refresh Seeq Workbench in your browser and take a look.

### Exporting

In Seeq Workbench, try changing the name of the _Example Analysis_ workbook to something like _My First Analysis Export_ so that you can tell that your changes get exported.

Now we will **search** for the workbooks we want to export. The syntax for a workbook search query is very similar to an item metadata search via `spy.search()`:

In [5]:
workbooks_df = spy.workbooks.search({
    'Path': 'My Import'
})

workbooks_df

,Count,Time
Results,2,00:00:00.09


,Archived,Created At,ID,Name,Owner ID,Owner Name,Owner Username,Path,Pinned,Search Folder ID,Type,Updated At,Workbook Type
0,0.0,2020-06-25 19:44:20.763774700+00:00,B70DAB96-988C-452E-9F91-3977802EE2D4,Example Analysis,45831CA7-46D9-4D1E-8CBB-4AAD8A7C5864,Agent API Key,agent_api_key,My Import,0.0,EC566060-7D5D-421A-870C-25A0E4F437D5,Workbook,2020-06-25 19:47:28.104959300+00:00,Analysis
1,0.0,2020-06-25 19:44:21.907146400+00:00,7EF60B3D-1836-4728-8A9E-6CFB9000B609,Example Topic,45831CA7-46D9-4D1E-8CBB-4AAD8A7C5864,Agent API Key,agent_api_key,My Import,0.0,EC566060-7D5D-421A-870C-25A0E4F437D5,Workbook,2020-06-25 19:47:28.351882200+00:00,Topic


As you can see, the `spy.workbooks.search()` command returns a metadata DataFrame with the properties of the workbooks. We can now use that to **pull**:

In [6]:
workbooks = spy.workbooks.pull(workbooks_df)
workbooks

,ID,Name,Workbook Type,Count,Time,Result
0,B70DAB96-988C-452E-9F91-3977802EE2D4,Example Analysis,Analysis,46,00:00:01.15,Success
1,7EF60B3D-1836-4728-8A9E-6CFB9000B609,Example Topic,Topic,6,00:00:00.25,Success


[Workbook "Example Analysis" (B70DAB96-988C-452E-9F91-3977802EE2D4),
 Workbook "Example Topic" (7EF60B3D-1836-4728-8A9E-6CFB9000B609)]

These are the same type of in-memory Python objects that we had when we executed `spy.workbooks.load()`. Now we can **save** them to disk:

In [7]:
if os.path.exists('../My First Export'):
    shutil.rmtree('../My First Export')
    
spy.workbooks.save(workbooks, '../My First Export')

In the parent folder of this documentation notebook, you'll find a new _My First Export_ folder that contains similar files to the _Example Export_ folder that's part of the documentation.

## Inspecting Worksheets

With the in-memory Python objects that result from `spy.workbooks.pull()` or `spy.workbooks.load()`, you can inspect the worksheets to see what is displayed on them. For example, let's look at what's in the Details Pane of the second worksheet of _Example Analysis_:

In [8]:
worksheet_items = workbooks[0].worksheets[1].display_items
worksheet_items

,Axis Align,Axis Auto Scale,Axis Group,Axis Max,Axis Min,Axis Show,Color,ID,Lane,Line Style,Line Width,Name,Samples Display,Type
0,Left,True,A,120.433655,66.778460,True,#068C45,3099FBFE-08D6-496F-80E5-A0F3460836B9,2.0,Solid,1.0,Area A_Temperature,Line,Signal
1,Left,True,A,120.433655,66.778460,True,#9D248F,29FD98E0-8618-4AAD-AA79-5B54DCC8CED6,2.0,Solid,2.5,Smooth Temperature,Line,Signal
2,Left,True,C,38.162385,0.002923,True,#CE561B,4B7DCF3A-8368-431C-8BD7-B2F4B34545CE,1.0,Solid,1.0,Area A_Compressor Power,Line,Signal
3,Left,True,D,96.598636,-100.000000,True,#00A2DD,4B2F16D8-364C-4D22-ADB2-AA6DCD934366,3.0,Solid,1.0,Area C_Temperature,Line,Signal
4,NaN,NaN,NaN,NaN,NaN,NaN,#4055A3,95A8C194-B94C-4397-9AE4-52E1739825AD,NaN,NaN,NaN,High Power,NaN,Condition
5,Left,True,A,120.433655,66.778460,True,#E1498E,959B57C9-00AB-41E2-8386-AF8F4BA9F427,2.0,Dash,1.0,Temperature Limit,NaN,Scalar


Now you can call `spy.pull()` to pull data for the items in the worksheet.

In [9]:
spy.pull(worksheet_items, start='2019-01-01T00:00:00', end='2019-01-02T00:00:00')

,ID,Name,Count,Time,Result
0,3099FBFE-08D6-496F-80E5-A0F3460836B9,Area A_Temperature,97,00:00:00.02,Success
1,29FD98E0-8618-4AAD-AA79-5B54DCC8CED6,Smooth Temperature,97,00:00:00.02,Success
2,4B7DCF3A-8368-431C-8BD7-B2F4B34545CE,Area A_Compressor Power,97,00:00:00.02,Success
3,4B2F16D8-364C-4D22-ADB2-AA6DCD934366,Area C_Temperature,97,00:00:00.02,Success
4,95A8C194-B94C-4397-9AE4-52E1739825AD,High Power,1,00:00:00.01,Success
5,959B57C9-00AB-41E2-8386-AF8F4BA9F427,Temperature Limit,1,00:00:00.00,Success


,Area A_Temperature,Smooth Temperature,Area A_Compressor Power,Area C_Temperature,High Power,Temperature Limit
2019-01-01 00:00:00-08:00,87.110200,86.931767,0.002923,67.714644,0.0,120.433655
2019-01-01 00:15:00-08:00,86.751221,86.647913,0.002923,67.444889,0.0,120.433655
2019-01-01 00:30:00-08:00,86.364014,86.396409,0.002923,67.422879,0.0,120.433655
2019-01-01 00:45:00-08:00,86.101156,86.187602,0.002924,67.475870,0.0,120.433655
2019-01-01 01:00:00-08:00,86.023508,86.010431,0.002923,67.367873,0.0,120.433655
2019-01-01 01:15:00-08:00,85.838905,85.814832,0.002923,67.332072,0.0,120.433655
2019-01-01 01:30:00-08:00,85.619200,85.533257,0.002923,67.209975,0.0,120.433655
2019-01-01 01:45:00-08:00,85.224636,85.122066,0.003023,67.541529,0.0,120.433655
2019-01-01 02:00:00-08:00,84.459696,84.595471,0.002923,66.808960,0.0,120.433655
2019-01-01 02:15:00-08:00,83.851354,84.018629,0.002923,66.946976,0.0,120.433655


Note that if you just wanted the full metadata for the items, you could execute `spy.search(worksheet_items[['ID']])`.

## Re-importing and Labels

If you **push** a set of workbooks more than once, then by default you will simply overwrite the existing workbooks with the saved content. This can be useful when you are "backing up" content to disk, perhaps for the purposes of version control.

You can choose to **push** and supply a _label_, which will create a separate copy of all of the imported items instead of modifying the existing ones. This is useful when you want to import something that you are iterating on prior to affecting the "published" version. For example, let's push our workbooks with the label of `In Development`:

In [10]:
spy.workbooks.push(workbooks, path='My Development Folder', label='In Development')

,ID,Name,Type,Workbook Type,Count,Time,Result,Pushed Workbook ID,URL
0,B70DAB96-988C-452E-9F91-3977802EE2D4,Example Analysis,Workbook,Analysis,51,00:00:01.23,Success,0723C538-0E74-471E-B130-5A9699809038,link
1,7EF60B3D-1836-4728-8A9E-6CFB9000B609,Example Topic,Workbook,Topic,6,00:00:00.15,Success,E767AD4C-0BF7-4EC6-87EE-040F44A7AE61,link


,ID,Name,Type,Workbook Type,Count,Time,Result,Pushed Workbook ID,URL
0,B70DAB96-988C-452E-9F91-3977802EE2D4,Example Analysis,Workbook,Analysis,51,0:00:01.230709,Success,0723C538-0E74-471E-B130-5A9699809038,http://localhost:34216/004D9716-3C7A-4E7D-9CC4...
1,7EF60B3D-1836-4728-8A9E-6CFB9000B609,Example Topic,Workbook,Topic,6,0:00:00.151597,Success,E767AD4C-0BF7-4EC6-87EE-040F44A7AE61,http://localhost:34216/004D9716-3C7A-4E7D-9CC4...


If you refresh Seeq Workbench, you'll notice that there is now a _My Development Folder_ and a separate copy of the Topic and Analysis that is independent of the original -- including all calculated items.

Pushing with the same value for the `label` argument will overwrite the content for that label. Change the label again if you want yet another separate copy.


## Importing to a Different Seeq Server

You may wish to copy content to a new/different Seeq Server by exporting and then importing. For example, you might have a _development_ server where you iterate on content and a _production_ server that you publish to when finished.

In order to accomplish this, you'll do one of two actions:

- If you're using the SPy module within Seeq Data Lab, you'll copy the exported folder to the other version of Seeq Data Lab and then push it from there.
- If you're using the SPy module with your own Python set up, you'll log in to the other server and push it.